In [10]:
# importing variables from json:
import json
with open('input.json', 'r') as f:
    input_data = json.load(f)

In [ ]:
!pip install keras-tuner matplotlib s

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import keras_tuner as kt

In [ ]:
# df = pd.read_csv(input_data['dataset_path1'])
df = pd.read_csv(input_data['dataset_path2'])

print("First 5 rows:")
display(df.head())

print("\nDataset shape:", df.shape)
print("\nBasic statistics:")
display(df.describe())

print("\nBasic Info:")
display(df.info())

In [ ]:
print("\nCorrelation Matrix:")
corr_matrix = df.corr(numeric_only=True)
display(corr_matrix)

In [ ]:
# plt.figure(figsize=(12,8))
# sns.heatmap(corr_matrix,
#             annot=True,
#             cmap='coolwarm'
#             # fmt=".2f",
#             # linewidths=.5)
# plt.title('Feature Correlation Matrix')
# plt.show()

plt.figure(figsize=(200, 160))  # Larger figure size
sns.heatmap(corr_matrix,
            # annot=False,  # Remove annotations to reduce clutter
            cmap='coolwarm',
            # linewidths=0.1,  # Add thin lines between cells
            linecolor='gray')
plt.title('Feature Correlation Matrix', fontsize=18)
plt.xticks(fontsize=10, rotation=45)  # Adjust x-axis labels
plt.yticks(fontsize=10, rotation=0)  # Adjust y-axis labels
plt.show()

In [ ]:
# Correlation Analysis
print("\n\033[1m1. Correlation Analysis\033[0m")
numeric_df = df.select_dtypes(include=[np.number])

# Pearson correlation
pearson_corr = numeric_df.corr()
print("\nPearson Correlation Matrix:")
display(pearson_corr)

# Spearman correlation (for non-linear relationships)
spearman_corr = numeric_df.corr(method='spearman')
print("\nSpearman Rank Correlation Matrix:")
display(spearman_corr)

In [ ]:
# Visualization
print("\n\033[1m2. Visual Relationships\033[0m")

# Heatmap
# plt.figure(figsize=(12,8))
# sns.heatmap(pearson_corr,
#             annot=True,
#             cmap='coolwarm',
#             vmin=-1, vmax=1,
#             mask=np.triu(np.ones_like(pearson_corr)))
# plt.title('Correlation Heatmap (Pearson)')
# plt.show()

# Pairplot for multivariate analysis
sns.pairplot(df.sample(min(200, len(df)), random_state=1))
plt.suptitle('Pairwise Relationships', y=1.02)
plt.show()

In [ ]:
# Statistical Testing
print("\n\033[1m3. Statistical Significance Testing\033[0m")

# For 2 numeric variables
if len(numeric_df.columns) >= 2:
    col1, col2 = numeric_df.columns[:2]
    r, p = stats.pearsonr(df[col1], df[col2])
    print(f"\nPearson test between {col1} and {col2}:")
    print(f"Correlation Coefficient: {r:.3f}")
    print(f"P-value: {p:.4f}")

In [ ]:
# Categorical Relationships
print("\n\033[1m4. Categorical Analysis\033[0m")

# For categorical vs numerical
categorical_cols = df.select_dtypes(exclude=[np.number]).columns
if len(categorical_cols) > 0 and len(numeric_df.columns) > 0:
    cat_var = categorical_cols[0]
    num_var = numeric_df.columns[0]

    plt.figure(figsize=(10,6))
    sns.boxplot(x=cat_var, y=num_var, data=df)
    plt.title(f'{cat_var} vs {num_var}')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
print("\n\033[1m5. Advanced Relationships\033[0m")

# Cross-correlation (lagged relationships)
if len(numeric_df.columns) >= 2:
    col1, col2 = numeric_df.columns[:2]
    cross_corr = np.correlate(df[col1], df[col2], mode='full')
    plt.plot(cross_corr)
    plt.title(f'Cross-correlation between {col1} and {col2}')
    plt.xlabel('Lag')
    plt.show()

# Model Training

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# data = pd.read_csv(input_data['dataset_path1'])
data = pd.read_csv(input_data['dataset_path2'])

X = data.drop("Target", axis=1).values
y = data["Target"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
# ### not scale x
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Normalize the target variable
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = y_scaler.transform(y_test.reshape(-1, 1)).flatten()

In [5]:
# # 2. Define the Sparse Autoencoder Model in TensorFlow/Keras
# input_dim = X_train.shape[1]
# latent_dim = 16  # tune this based on dataset
# #### hpo here

# #input
# input_layer = keras.Input(shape=(input_dim,))

# # Encoder
# encoded = layers.Dense(64, activation='relu')(input_layer)
# encoded = layers.BatchNormalization()(encoded)
# encoded = layers.Dropout(0.2)(encoded)
# latent = layers.Dense(latent_dim, activation='linear',
#                       activity_regularizer=regularizers.l1(1e-3))(encoded)
# ### ## activity_regularizer, lr, do HPO,
# ### start final report

# # Decoder
# decoded = layers.Dense(64, activation='relu')(latent)
# decoded = layers.BatchNormalization()(decoded)
# decoded = layers.Dropout(0.2)(decoded)
# decoded = layers.Dense(input_dim, activation='linear')(decoded)

# autoencoder = keras.Model(inputs=input_layer, outputs=decoded, name="sparse_autoencoder")
# autoencoder.compile(optimizer='adam', loss='mse')
# autoencoder.summary()
#

In [ ]:
# TRY DECREASE LAYER, Python class, plotting everything(loss,lr,acc, etc),
# activity_regularizer l1 play with it and relation with lr, powerpoint to it,
# Add Sparse

def build_model(hp):
    input_dim = X_train.shape[1]  # Number of input features

    # Hyperparameters to tune:
    encoder_units = hp.Int('encoder_units', min_value=8, max_value=256, step=2)
    decoder_units = hp.Int('decoder_units', min_value=8, max_value=256, step=2)
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.8, step=0.1)
    latent_dim = hp.Int('latent_dim', min_value=2, max_value=128, step=2)
    l1_reg = hp.Float('l1_reg', min_value=1e-5, max_value=1e-2, sampling='log')
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    # Input layer
    input_layer = keras.Input(shape=(input_dim,))

    # Encoder
    x = layers.Dense(encoder_units, activation='relu')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    latent = layers.Dense(latent_dim, activation='linear', activity_regularizer=regularizers.l1(l1_reg), name='latent')(x)

    # Decoder
    x = layers.Dense(decoder_units, activation='relu')(latent)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)
    output_layer = layers.Dense(input_dim, activation='linear')(x)

    # Define the autoencoder model
    model = keras.Model(inputs=input_layer, outputs=output_layer, name="sparse_autoencoder")

    # Compile the model with the tunable learning rate
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mse')

    return model

#2
# def build_model(hp):
#     input_dim = X_train.shape[1]  # Number of input features

#     # Hyperparameters to tune:
#     encoder_units_1 = hp.Int('encoder_units_1', min_value=8, max_value=256, step=2)
#     encoder_units_2 = hp.Int('encoder_units_2', min_value=8, max_value=256, step=2)
#     decoder_units_1 = hp.Int('decoder_units_1', min_value=8, max_value=256, step=2)
#     decoder_units_2 = hp.Int('decoder_units_2', min_value=8, max_value=256, step=2)
#     dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.8, step=0.1)
#     latent_dim = hp.Int('latent_dim', min_value=2, max_value=128, step=2)
#     l1_reg = hp.Float('l1_reg', min_value=1e-5, max_value=1e-2, sampling='log')
#     learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

#     # Input layer
#     input_layer = keras.Input(shape=(input_dim,))

#     # Encoder: Increased layers
#     x = layers.Dense(encoder_units_1, activation='relu')(input_layer)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     x = layers.Dense(encoder_units_2, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     latent = layers.Dense(latent_dim, activation='linear',
#                           activity_regularizer=regularizers.l1(l1_reg),
#                           name='latent')(x)

#     # Decoder: Increased layers
#     x = layers.Dense(decoder_units_1, activation='relu')(latent)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     x = layers.Dense(decoder_units_2, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     output_layer = layers.Dense(input_dim, activation='linear')(x)

#     # Define and compile the autoencoder model
#     model = keras.Model(inputs=input_layer, outputs=output_layer, name="sparse_autoencoder")
#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
#                   loss='mse')

#     return model


# Set up the Keras Tuner with Random Search (other tuners like BayesianOptimization are also available)
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=50,           # Number of hyperparameter combinations to try
    executions_per_trial=1,  # Number of models to build for each trial
    directory='hpo_dir',
    project_name='sparse_autoencoder'
)

# Print a summary of the search space
tuner.search_space_summary()

# Run the hyperparameter search.
# Here we train the model as an autoencoder (input is the same as output).
tuner.search(X_train, X_train, epochs=500, validation_split=0.1)

# Retrieve the best model found by the tuner
best_model = tuner.get_best_models(num_models=1)[0]

# Display a summary of the best model architecture
best_model.summary()


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Build the autoencoder with the best hyperparameters and train it
autoencoder = best_model
history = autoencoder.fit(X_train, X_train,
                          epochs=500,
                          batch_size=8,
                          validation_split=0.1,
                          callbacks=[early_stop],
                          verbose=1)

# Confirm that it’s a Sparse Autoencoder (the latent layer includes an L1 regularizer)
print("Sparse Autoencoder defined with L1 regularization on the latent layer (named 'latent').")

In [ ]:
# 4. Create the Encoder Model for Feature Extraction
# Create a model to extract the latent features
encoder = keras.Model(inputs=best_model.input, outputs=best_model.get_layer("latent").output, name="encoder")
latent_train = encoder.predict(X_train)
latent_test = encoder.predict(X_test)

In [ ]:
# 5. Build and Train a Downstream Regression Model for AGB Prediction
def build_regressor_model(hp):
    reg_input = keras.Input(shape=(latent_train.shape[1],))

    units_1 = hp.Int('units_1', min_value=16, max_value=128, step=16, default=32)
    units_2 = hp.Int('units_2', min_value=8, max_value=64, step=8, default=16)
    dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1, default=0.2)
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log', default=1e-3)

    x = layers.Dense(units_1, activation='relu')(reg_input)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(units_2, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rate)(x)

    reg_output = layers.Dense(1, activation='linear')(x)

    regressor = keras.Model(inputs=reg_input, outputs=reg_output, name="regressor")

    regressor.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='mse', metrics=['mae'])
    return regressor

#2
# def build_regressor_model(hp):
#     reg_input = keras.Input(shape=(latent_train.shape[1],))

#     # Hyperparameters for layer sizes
#     units_1 = hp.Int('units_1', min_value=32, max_value=256, step=32, default=64)
#     units_2 = hp.Int('units_2', min_value=16, max_value=128, step=16, default=32)
#     units_3 = hp.Int('units_3', min_value=8, max_value=64, step=8, default=16)
#     # Optionally add even another layer:
#     # units_4 = hp.Int('units_4', min_value=8, max_value=64, step=8, default=16)

#     dropout_rate = hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1, default=0.2)
#     learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log', default=1e-3)

#     # First hidden layer
#     x = layers.Dense(units_1, activation='relu')(reg_input)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     # Second hidden layer
#     x = layers.Dense(units_2, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     # Third hidden layer (new layer)
#     x = layers.Dense(units_3, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(dropout_rate)(x)

#     reg_output = layers.Dense(1, activation='linear')(x)

#     regressor = keras.Model(inputs=reg_input, outputs=reg_output, name="regressor")

#     regressor.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
#                       loss='mse', metrics=['mae'])
#     return regressor


# Then set up the tuner:
tuner_regressor = kt.RandomSearch(
    build_regressor_model,
    objective='val_loss',
    max_trials=50,
    executions_per_trial=1,
    directory='hpo_regressor_dir2',
    project_name='regressor_agb'
)

tuner_regressor.search(latent_train, y_train, epochs=300, validation_split=0.1)

best_regressor = tuner_regressor.get_best_models(num_models=1)[0]
best_regressor.summary()


reg_epochs = 500
best_regressor.fit(latent_train, y_train,
              epochs=reg_epochs,
              batch_size=8,
              validation_split=0.1,
              verbose=1)

In [ ]:
y_pred_scaled = best_regressor.predict(latent_test).flatten()
# Inverse transform predictions and true values to get original scale
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

mse_value = mean_squared_error(y_test_inv, y_pred)
r2 = r2_score(y_test_inv, y_pred)
print(f"Test MSE for AGB prediction: {mse_value:.4f}")
print(f"Test R^2 for AGB prediction: {r2:.4f}")

In [ ]:
# make graphs

In [ ]:
# Plot and save the Autoencoder training loss history
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Autoencoder Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig('autoencoder_loss_history.png')
plt.show()
print("Autoencoder loss history plot saved as 'autoencoder_loss_history.png'.")

# Plot and save the predicted vs. actual AGB scatter plot for the regressor
plt.figure(figsize=(8,6))
plt.scatter(y_test_inv, y_pred, color='blue', alpha=0.6, label='Predictions')
plt.plot([min(y_test_inv), max(y_test_inv)], [min(y_test_inv), max(y_test_inv)], 'r--', label='Ideal Fit')
plt.xlabel('Actual AGB')
plt.ylabel('Predicted AGB')
plt.title(f'Predicted vs Actual AGB (R^2 = {r2:.2f})')
plt.legend()
plt.grid(True)
plt.savefig('predicted_vs_actual.png')
plt.show()
print("Predicted vs Actual AGB plot saved as 'predicted_vs_actual.png'.")

# Residual plot
residuals = y_test_inv - y_pred

plt.figure(figsize=(10, 6))
sns.residplot(x=y_pred, y=residuals, lowess=True, scatter_kws={"s": 50, "alpha": 0.5}, line_kws={"color": "red", "lw": 2})
plt.xlabel('Predicted AGB')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Above Ground Biomass')
plt.grid(True)
plt.show()

import scipy.stats as stats

# Q-Q plot
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.grid(True)
plt.show()

from sklearn.model_selection import learning_curve

# # Learning curve
# train_sizes, train_scores, val_scores = learning_curve(
#     best_regressor, latent_train, y_train, cv=5, scoring='neg_mean_squared_error',
#     train_sizes=np.linspace(0.1, 1.0, 10)
# )

# train_scores_mean = -train_scores.mean(axis=1)
# val_scores_mean = -val_scores.mean(axis=1)

# plt.figure(figsize=(10, 6))
# plt.plot(train_sizes, train_scores_mean, 'o-', color='blue', label='Training error')
# plt.plot(train_sizes, val_scores_mean, 'o-', color='red', label='Validation error')
# plt.xlabel('Training Set Size')
# plt.ylabel('Mean Squared Error')
# plt.title('Learning Curve')
# plt.legend(loc='best')
# plt.grid(True)
# plt.show()

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Regression metrics
mae = mean_absolute_error(y_test_inv, y_pred)
mse = mean_squared_error(y_test_inv, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_inv, y_pred)

print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'R-squared (R2): {r2:.2f}')

# Feature importance (example for tree-based models)
# importances = best_regressor.feature_importances_
# feature_names = data.columns[:-1]  # Assuming the last column is the target

# plt.figure(figsize=(10, 6))
# sns.barplot(x=importances, y=feature_names)
# plt.xlabel('Importance')
# plt.ylabel('Feature')
# plt.title('Feature Importance')
# plt.grid(True)
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.inspection import permutation_importance
import numpy as np

# 1. Training History Visualization
def plot_training_history(history, title):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    if 'mae' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['mae'], label='Train MAE')
        plt.plot(history.history['val_mae'], label='Validation MAE')
        plt.title(f'{title} - MAE')
        plt.ylabel('MAE')
        plt.xlabel('Epoch')
        plt.legend()

    plt.tight_layout()
    plt.savefig(f'{title}_training_history.png')
    plt.show()

# Plot autoencoder training history
plot_training_history(history, "Autoencoder")

# Plot regressor training history (after training best_regressor)
regressor_history = best_regressor.fit(latent_train, y_train,
                                      epochs=reg_epochs,
                                      batch_size=32,
                                      validation_split=0.1,
                                      verbose=1)
plot_training_history(regressor_history, "Regressor")

# 2. Latent Space Visualization
def visualize_latent_space(latent_rep, y, title):
    plt.figure(figsize=(12, 5))

    # PCA Visualization
    plt.subplot(1, 2, 1)
    pca = PCA(n_components=2)
    latent_pca = pca.fit_transform(latent_rep)
    plt.scatter(latent_pca[:, 0], latent_pca[:, 1], c=y, cmap='viridis', alpha=0.6)
    plt.colorbar(label='Target Value')
    plt.title(f'{title} - PCA Projection')

    # t-SNE Visualization
    plt.subplot(1, 2, 2)
    tsne = TSNE(n_components=2, perplexity=30)
    latent_tsne = tsne.fit_transform(latent_rep)
    plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], c=y, cmap='viridis', alpha=0.6)
    plt.colorbar(label='Target Value')
    plt.title(f'{title} - t-SNE Projection')

    plt.tight_layout()
    plt.savefig(f'{title}_latent_space.png')
    plt.show()

visualize_latent_space(latent_train, y_train, "Training_Latent_Space")

# 3. Prediction Visualization
plt.figure(figsize=(10, 6))
plt.scatter(y_test_inv, y_pred, alpha=0.6)
plt.plot([min(y_test_inv), max(y_test_inv)],
         [min(y_test_inv), max(y_test_inv)], 'r--')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title(f'Actual vs Predicted Values (R² = {r2:.2f})')
plt.grid(True)
plt.savefig('actual_vs_predicted.png')
plt.show()

# 4. Residual Plot
residuals = y_test_inv - y_pred
plt.figure(figsize=(10, 6))
sns.residplot(x=y_pred, y=residuals, lowess=True,
             line_kws={'color': 'red', 'lw': 1})
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Analysis')
plt.grid(True)
plt.savefig('residual_plot.png')
plt.show()

# # 5. Feature Importance (Permutation Importance)
# from sklearn.metrics import mean_squared_error

# result = permutation_importance(
#     best_regressor, latent_test, y_test_inv,
#     n_repeats=10, random_state=42,
#     scoring='neg_mean_squared_error'
# )


# plt.figure(figsize=(10, 6))
# plt.boxplot(result.importances[sorted_idx].T,
#             vert=False, labels=np.arange(latent_train.shape[1])[sorted_idx])
# plt.title("Permutation Importance (Latent Features)")
# plt.xlabel("Importance")
# plt.ylabel("Latent Feature Index")
# plt.savefig('feature_importance.png')
# plt.show()

# 6. Model Architecture Visualization
from tensorflow.keras.utils import plot_model

# Visualize autoencoder
plot_model(best_model, to_file='autoencoder_architecture.png',
          show_shapes=True, show_layer_names=True)

# Visualize regressor
plot_model(best_regressor, to_file='regressor_architecture.png',
          show_shapes=True, show_layer_names=True)

# 7. Hyperparameter Optimization Results
def plot_hpo_results(tuner):
    results = tuner.oracle.get_best_trials(num_trials=10)
    hp_names = list(tuner.get_best_hyperparameters()[0].values.keys())

    plt.figure(figsize=(15, 8))
    for i, hp_name in enumerate(hp_names):
        plt.subplot(2, 4, i+1)
        values = [t.hyperparameters.values[hp_name] for t in results]
        losses = [t.score for t in results]
        plt.scatter(values, losses)
        plt.title(hp_name)
        plt.xlabel('Value')
        plt.ylabel('Validation Loss')

    plt.tight_layout()
    plt.savefig('hpo_results.png')
    plt.show()

plot_hpo_results(tuner)
plot_hpo_results(tuner_regressor)